In [2]:
import tensorflow as tf
from tensorflow import data
import shutil
import math
from datetime import datetime
from tensorflow.python.feature_column import feature_column

from tensorflow.contrib.learn import learn_runner
from tensorflow.contrib.learn import make_export_strategy

print(tf.__version__)

1.4.0


## Steps to use the TF Estimator APIs
1. Define dataset **metadata**
2. Define **data input function** to read the data from the source (.tfrecords) + **apply pre-processing**
3. Create TF **feature columns** based on metadata + **extended feature columns**
4. Instantiate an **estimator** (LinearCombinedDNN) with the required **feature columns (wide/deep) & parameters**
5. Define a **serving function**
6. Run **Experiment** by supplying training and validation data, as well as required parameters
7. **Evaluate** the model using test data
8. Perform **predictions**

In [2]:
train_data_files = ['data/train-data.tfrecords']
valid_data_files = ['data/valid-data.tfrecords']
test_data_files = ['data/test-data.tfrecords']

model_name = 'reg-model-05'

resume = False
train = True
preprocess = True

## 1. Define Dataset Metadata

In [3]:
HEADER = ['key','x','y','alpha','beta','target']
HEADER_DEFAULTS = [[0], [0.0], [0.0], ['NA'], ['NA'], [0.0]]

NUMERIC_FEATURE_NAMES = ['x', 'y']  

CATEGORICAL_FEATURE_NAMES_WITH_VOCABULARY = {'alpha':['ax01', 'ax02'], 'beta':['bx01', 'bx02']}
CATEGORICAL_FEATURE_NAMES = list(CATEGORICAL_FEATURE_NAMES_WITH_VOCABULARY.keys())

FEATURE_NAMES = NUMERIC_FEATURE_NAMES + CATEGORICAL_FEATURE_NAMES

TARGET_NAME = 'target'

UNUSED_FEATURE_NAMES = list(set(HEADER) - set(FEATURE_NAMES) - {TARGET_NAME})

print("Header: {}".format(HEADER))
print("Numeric Features: {}".format(NUMERIC_FEATURE_NAMES))
print("Categorical Features: {}".format(CATEGORICAL_FEATURE_NAMES))
print("Target: {}".format(TARGET_NAME))
print("Unused Features: {}".format(UNUSED_FEATURE_NAMES))

Header: ['key', 'x', 'y', 'alpha', 'beta', 'target']
Numeric Features: ['x', 'y']
Categorical Features: ['alpha', 'beta']
Target: target
Unused Features: ['key']


## 2. Define Data Input Function

### a. parsing and preprocessing logic

In [4]:
def parse_tf_example(example_proto):

    feature_spec = {}

    for feature_name in NUMERIC_FEATURE_NAMES:
        feature_spec[feature_name] = tf.FixedLenFeature(shape=(1), dtype=tf.float32)
    
    for feature_name in CATEGORICAL_FEATURE_NAMES:
        feature_spec[feature_name] = tf.FixedLenFeature(shape=(1), dtype=tf.string)
    
    feature_spec[TARGET_NAME] = tf.FixedLenFeature(shape=(1), dtype=tf.float32)

    parsed_features = tf.parse_example(serialized=example_proto, features=feature_spec)
    
    target = parsed_features.pop(TARGET_NAME)
    
    return parsed_features, target


def process_features(features):
    
    if preprocess:
        features["x_2"] = tf.square(features['x'])
        features["y_2"] = tf.square(features['y'])
        features["xy"] = tf.multiply(features['x'], features['y']) # features['x'] * features['y']
        features['dist_xy'] =  tf.sqrt(tf.squared_difference(features['x'],features['y']))
    
    return features

### b. data pipeline input function

In [5]:
def tfrecods_input_fn(file_names, mode=tf.estimator.ModeKeys.EVAL, 
                 num_epochs=None, 
                 batch_size=200):
    
    shuffle = True if mode == tf.estimator.ModeKeys.TRAIN else False

    dataset = data.TFRecordDataset(filenames=file_names)

    if shuffle:
        dataset = dataset.shuffle(buffer_size=2 * batch_size + 1)
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(lambda tf_example: parse_tf_example(tf_example))
    dataset = dataset.map(lambda features, target: (process_features(features), target))
    dataset = dataset.repeat(num_epochs)
    iterator = dataset.make_one_shot_iterator()
    
    features, target = iterator.get_next()
    return features, target

In [6]:
features, target = tfrecods_input_fn(file_names=train_data_files)
print("Feature read from TFRecords: {}".format(list(features.keys())))
print("Target read from TFRecords: {}".format(target))

Feature read from TFRecords: ['alpha', 'beta', 'dist_xy', 'x', 'x_2', 'xy', 'y', 'y_2']
Target read from TFRecords: Tensor("IteratorGetNext:8", shape=(?, 1), dtype=float32)


## 3. Define Feature Columns

In [7]:
def get_feature_columns():
    
    CONSTRUCTED_NUMERIC_FEATURES_NAMES = ['x_2', 'y_2', 'xy', 'dist_xy']
    all_numeric_feature_names = NUMERIC_FEATURE_NAMES.copy() 
    
    if preprocess:
        all_numeric_feature_names += CONSTRUCTED_NUMERIC_FEATURES_NAMES

    numeric_columns = {feature_name: tf.feature_column.numeric_column(feature_name)
                       for feature_name in all_numeric_feature_names}

    categorical_column_with_vocabulary = \
        {item[0]: tf.feature_column.categorical_column_with_vocabulary_list(item[0], item[1])
         for item in CATEGORICAL_FEATURE_NAMES_WITH_VOCABULARY.items()}
        
    feature_columns = {}

    if numeric_columns is not None:
        feature_columns.update(numeric_columns)

    if categorical_column_with_vocabulary is not None:
        feature_columns.update(categorical_column_with_vocabulary)

    # apply feature extentions
    
    feature_columns['alpha_X_beta'] = tf.feature_column.crossed_column(
        [feature_columns['alpha'], feature_columns['beta']], 4)
    
    return feature_columns

feature_columns = get_feature_columns()
print("Feature Columns: {}".format(feature_columns))

Feature Columns: {'x': _NumericColumn(key='x', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), 'y': _NumericColumn(key='y', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), 'x_2': _NumericColumn(key='x_2', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), 'y_2': _NumericColumn(key='y_2', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), 'xy': _NumericColumn(key='xy', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), 'dist_xy': _NumericColumn(key='dist_xy', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), 'alpha': _VocabularyListCategoricalColumn(key='alpha', vocabulary_list=('ax01', 'ax02'), dtype=tf.string, default_value=-1, num_oov_buckets=0), 'beta': _VocabularyListCategoricalColumn(key='beta', vocabulary_list=('bx01', 'bx02'), dtype=tf.string, default_value=-1, num_oov_buckets=0), 'alpha_X_beta': _CrossedColumn(keys=(_VocabularyListCategoricalColumn(key=

## 4. Instantiate an Estimator

In [8]:
def create_estimator(run_config, hparams):
    
    feature_columns = list(get_feature_columns().values())
    
    dense_columns = list(
        filter(lambda column: isinstance(column, feature_column._NumericColumn),
               feature_columns
        )
    )

    categorical_columns = list(
        filter(lambda column: isinstance(column, feature_column._VocabularyListCategoricalColumn) |
                              isinstance(column, feature_column._BucketizedColumn),
                   feature_columns)
    )

    indicator_columns = list(
            map(lambda column: tf.feature_column.indicator_column(column),
                categorical_columns)
    )
    
    estimator = tf.estimator.DNNLinearCombinedRegressor(
        
        dnn_feature_columns= dense_columns + indicator_columns,
        linear_feature_columns = categorical_columns,
        
        dnn_hidden_units= hparams.hidden_units,
        
        dnn_optimizer= tf.train.AdamOptimizer(),
        
        dnn_activation_fn= tf.nn.elu,
        dnn_dropout= hparams.dropout_prob,
        
        config= run_config
    )
    
    return estimator

## 5. Define Serving Funcion

In [9]:
def csv_serving_input_fn():
    
    SERVING_HEADER = ['x','y','alpha','beta']
    SERVING_HEADER_DEFAULTS = [[0.0], [0.0], ['NA'], ['NA']]

    rows_string_tensor = tf.placeholder(dtype=tf.string,
                                         shape=[None],
                                         name='rows_string_tensor')
    
    receiver_tensor = {'rows': rows_string_tensor}

    row_columns = tf.expand_dims(rows_string_tensor, -1)
    columns = tf.decode_csv(row_columns, record_defaults=SERVING_HEADER_DEFAULTS)
    features = dict(zip(SERVING_HEADER, columns))

    return tf.estimator.export.ServingInputReceiver(
        process_features(features), receiver_tensor)

## 6. Run Experiment

### a. Define Experiment Function

In [10]:
def generate_experiment_fn(**experiment_args):

    def _experiment_fn(run_config, hparams):

        train_input_fn = lambda: tfrecods_input_fn(
            train_data_files,
            mode = tf.contrib.learn.ModeKeys.TRAIN,
            num_epochs=hparams.num_epochs,
            batch_size=hparams.batch_size
        )

        eval_input_fn = lambda: tfrecods_input_fn(
            valid_data_files,
            mode=tf.contrib.learn.ModeKeys.EVAL,
            batch_size=hparams.batch_size
        )

        estimator = create_estimator(run_config, hparams)

        return tf.contrib.learn.Experiment(
            estimator,
            train_input_fn=train_input_fn,
            eval_input_fn=eval_input_fn,
            **experiment_args
        )

    return _experiment_fn

### b. Set HParam and RunConfig

In [11]:
hparams  = tf.contrib.training.HParams(
    num_epochs = 10000,
    batch_size = 500,
    hidden_units=[16, 12, 8], 
    dropout_prob = 0.01)

model_dir = 'trained_models/{}'.format(model_name)

run_config = tf.contrib.learn.RunConfig(
            model_dir=model_dir
)

print(run_config.model_dir)

trained_models/reg-model-05


### c. Run Experiment via learn_runner

In [12]:
if not resume:
    print("Removing previous artifacts...")
    shutil.rmtree(model_dir, ignore_errors=True)
else:
    print("Resuming training...") 

if train:
    tf.logging.set_verbosity(tf.logging.INFO)
    
    print("")
    print("Starting experiment...") 
    print("") 
    
    learn_runner.run(
        experiment_fn=generate_experiment_fn(

            export_strategies=[make_export_strategy(
                csv_serving_input_fn,
                exports_to_keep=1
            )]
        ),
        run_config=run_config,
        schedule="train_and_evaluate",
        hparams=hparams
    )
    
    print("") 
    print("experiment finished...") 

Removing previous artifacts...

Starting experiment...

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11b8d4ba8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'trained_models/reg-model-05'}
Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into trained_models/reg-model-05/model.ckpt.
INFO:tensorflow:Starting evaluation at 2017-10-31-08:55:30
INFO:tensorflow:Rest

INFO:tensorflow:loss = 52412.2, step = 2801 (0.765 sec)
INFO:tensorflow:global_step/sec: 125.577
INFO:tensorflow:loss = 38609.2, step = 2901 (0.795 sec)
INFO:tensorflow:global_step/sec: 135.082
INFO:tensorflow:loss = 54989.0, step = 3001 (0.743 sec)
INFO:tensorflow:global_step/sec: 118.654
INFO:tensorflow:loss = 40283.8, step = 3101 (0.841 sec)
INFO:tensorflow:global_step/sec: 107.592
INFO:tensorflow:loss = 54039.0, step = 3201 (0.930 sec)
INFO:tensorflow:global_step/sec: 115.476
INFO:tensorflow:loss = 41787.2, step = 3301 (0.864 sec)
INFO:tensorflow:global_step/sec: 149.583
INFO:tensorflow:loss = 37513.7, step = 3401 (0.667 sec)
INFO:tensorflow:global_step/sec: 182.994
INFO:tensorflow:loss = 42552.0, step = 3501 (0.546 sec)
INFO:tensorflow:global_step/sec: 192.42
INFO:tensorflow:loss = 42498.4, step = 3601 (0.520 sec)
INFO:tensorflow:global_step/sec: 192.519
INFO:tensorflow:loss = 48185.8, step = 3701 (0.519 sec)
INFO:tensorflow:global_step/sec: 192.561
INFO:tensorflow:loss = 35607.5,

INFO:tensorflow:global_step/sec: 174.339
INFO:tensorflow:loss = 41673.8, step = 11301 (0.575 sec)
INFO:tensorflow:global_step/sec: 172.287
INFO:tensorflow:loss = 43931.6, step = 11401 (0.579 sec)
INFO:tensorflow:global_step/sec: 188.236
INFO:tensorflow:loss = 54849.5, step = 11501 (0.531 sec)
INFO:tensorflow:global_step/sec: 189.39
INFO:tensorflow:loss = 42901.4, step = 11601 (0.528 sec)
INFO:tensorflow:global_step/sec: 189.68
INFO:tensorflow:loss = 49361.4, step = 11701 (0.528 sec)
INFO:tensorflow:global_step/sec: 181.091
INFO:tensorflow:loss = 40122.0, step = 11801 (0.552 sec)
INFO:tensorflow:global_step/sec: 170.474
INFO:tensorflow:loss = 48637.8, step = 11901 (0.587 sec)
INFO:tensorflow:global_step/sec: 177.111
INFO:tensorflow:loss = 49511.7, step = 12001 (0.564 sec)
INFO:tensorflow:global_step/sec: 181.615
INFO:tensorflow:loss = 44869.6, step = 12101 (0.551 sec)
INFO:tensorflow:global_step/sec: 179.844
INFO:tensorflow:loss = 52426.3, step = 12201 (0.556 sec)
INFO:tensorflow:global

INFO:tensorflow:global_step/sec: 193
INFO:tensorflow:loss = 41285.1, step = 19701 (0.518 sec)
INFO:tensorflow:global_step/sec: 191.321
INFO:tensorflow:loss = 39843.9, step = 19801 (0.523 sec)
INFO:tensorflow:global_step/sec: 182.692
INFO:tensorflow:loss = 50947.4, step = 19901 (0.547 sec)
INFO:tensorflow:global_step/sec: 183.508
INFO:tensorflow:loss = 41951.8, step = 20001 (0.545 sec)
INFO:tensorflow:global_step/sec: 191.914
INFO:tensorflow:loss = 34881.1, step = 20101 (0.521 sec)
INFO:tensorflow:global_step/sec: 191.385
INFO:tensorflow:loss = 41976.0, step = 20201 (0.522 sec)
INFO:tensorflow:global_step/sec: 176.582
INFO:tensorflow:loss = 45522.9, step = 20301 (0.568 sec)
INFO:tensorflow:global_step/sec: 170.087
INFO:tensorflow:loss = 36734.6, step = 20401 (0.587 sec)
INFO:tensorflow:global_step/sec: 181.116
INFO:tensorflow:loss = 36380.8, step = 20501 (0.553 sec)
INFO:tensorflow:global_step/sec: 169.007
INFO:tensorflow:loss = 46207.7, step = 20601 (0.591 sec)
INFO:tensorflow:global_s

INFO:tensorflow:global_step/sec: 190.619
INFO:tensorflow:loss = 48274.3, step = 28101 (0.525 sec)
INFO:tensorflow:global_step/sec: 192.406
INFO:tensorflow:loss = 43030.1, step = 28201 (0.519 sec)
INFO:tensorflow:global_step/sec: 192.941
INFO:tensorflow:loss = 31433.1, step = 28301 (0.518 sec)
INFO:tensorflow:global_step/sec: 192.681
INFO:tensorflow:loss = 31556.0, step = 28401 (0.519 sec)
INFO:tensorflow:global_step/sec: 191.394
INFO:tensorflow:loss = 48698.9, step = 28501 (0.523 sec)
INFO:tensorflow:global_step/sec: 191.411
INFO:tensorflow:loss = 43509.1, step = 28601 (0.522 sec)
INFO:tensorflow:global_step/sec: 173.795
INFO:tensorflow:loss = 53132.7, step = 28701 (0.577 sec)
INFO:tensorflow:global_step/sec: 161.01
INFO:tensorflow:loss = 55666.9, step = 28801 (0.621 sec)
INFO:tensorflow:global_step/sec: 168.508
INFO:tensorflow:loss = 41180.0, step = 28901 (0.594 sec)
INFO:tensorflow:global_step/sec: 149.54
INFO:tensorflow:loss = 45414.0, step = 29001 (0.668 sec)
INFO:tensorflow:global

...

INFO:tensorflow:loss = 36049.7, step = 203201 (0.594 sec)
INFO:tensorflow:global_step/sec: 177.458
INFO:tensorflow:loss = 38103.1, step = 203301 (0.565 sec)
INFO:tensorflow:global_step/sec: 120.612
INFO:tensorflow:loss = 47058.0, step = 203401 (0.834 sec)
INFO:tensorflow:global_step/sec: 128.058
INFO:tensorflow:loss = 34902.0, step = 203501 (0.777 sec)
INFO:tensorflow:global_step/sec: 156.414
INFO:tensorflow:loss = 49144.2, step = 203601 (0.636 sec)
INFO:tensorflow:global_step/sec: 171.197
INFO:tensorflow:loss = 51421.8, step = 203701 (0.590 sec)
INFO:tensorflow:global_step/sec: 119.99
INFO:tensorflow:loss = 42406.5, step = 203801 (0.828 sec)
INFO:tensorflow:global_step/sec: 148.689
INFO:tensorflow:loss = 47036.4, step = 203901 (0.673 sec)
INFO:tensorflow:global_step/sec: 168.486
INFO:tensorflow:loss = 48217.6, step = 204001 (0.597 sec)
INFO:tensorflow:global_step/sec: 168.907
INFO:tensorflow:loss = 39720.8, step = 204101 (0.589 sec)
INFO:tensorflow:global_step/sec: 179.926
INFO:tensor

INFO:tensorflow:loss = 45998.6, step = 211501 (0.725 sec)
INFO:tensorflow:global_step/sec: 126.101
INFO:tensorflow:loss = 41958.4, step = 211601 (0.794 sec)
INFO:tensorflow:global_step/sec: 139.707
INFO:tensorflow:loss = 35622.2, step = 211701 (0.714 sec)
INFO:tensorflow:global_step/sec: 146.978
INFO:tensorflow:loss = 47870.4, step = 211801 (0.680 sec)
INFO:tensorflow:global_step/sec: 160.11
INFO:tensorflow:loss = 39573.9, step = 211901 (0.626 sec)
INFO:tensorflow:global_step/sec: 155.991
INFO:tensorflow:loss = 42118.0, step = 212001 (0.640 sec)
INFO:tensorflow:global_step/sec: 147.427
INFO:tensorflow:loss = 41229.0, step = 212101 (0.682 sec)
INFO:tensorflow:global_step/sec: 159.242
INFO:tensorflow:loss = 39816.3, step = 212201 (0.628 sec)
INFO:tensorflow:global_step/sec: 131.072
INFO:tensorflow:loss = 42938.3, step = 212301 (0.761 sec)
INFO:tensorflow:global_step/sec: 159.912
INFO:tensorflow:loss = 47217.2, step = 212401 (0.624 sec)
INFO:tensorflow:global_step/sec: 176.384
INFO:tensor

INFO:tensorflow:loss = 36285.9, step = 219801 (0.606 sec)
INFO:tensorflow:global_step/sec: 165.068
INFO:tensorflow:loss = 48474.3, step = 219901 (0.607 sec)
INFO:tensorflow:global_step/sec: 158.459
INFO:tensorflow:loss = 35927.4, step = 220001 (0.631 sec)
INFO:tensorflow:global_step/sec: 161.71
INFO:tensorflow:loss = 43954.4, step = 220101 (0.617 sec)
INFO:tensorflow:global_step/sec: 174.1
INFO:tensorflow:loss = 58199.2, step = 220201 (0.575 sec)
INFO:tensorflow:global_step/sec: 162.01
INFO:tensorflow:loss = 37724.2, step = 220301 (0.619 sec)
INFO:tensorflow:global_step/sec: 164.058
INFO:tensorflow:loss = 46463.3, step = 220401 (0.607 sec)
INFO:tensorflow:global_step/sec: 178.116
INFO:tensorflow:loss = 52689.0, step = 220501 (0.561 sec)
INFO:tensorflow:global_step/sec: 171.148
INFO:tensorflow:loss = 42403.8, step = 220601 (0.586 sec)
INFO:tensorflow:global_step/sec: 151.108
INFO:tensorflow:loss = 40884.6, step = 220701 (0.661 sec)
INFO:tensorflow:global_step/sec: 175.099
INFO:tensorflo

INFO:tensorflow:loss = 39626.7, step = 228101 (0.549 sec)
INFO:tensorflow:global_step/sec: 181.578
INFO:tensorflow:loss = 38906.5, step = 228201 (0.551 sec)
INFO:tensorflow:global_step/sec: 175.396
INFO:tensorflow:loss = 46024.9, step = 228301 (0.570 sec)
INFO:tensorflow:global_step/sec: 177.433
INFO:tensorflow:loss = 41623.4, step = 228401 (0.563 sec)
INFO:tensorflow:global_step/sec: 180.941
INFO:tensorflow:loss = 43723.0, step = 228501 (0.552 sec)
INFO:tensorflow:global_step/sec: 178.617
INFO:tensorflow:loss = 36431.3, step = 228601 (0.561 sec)
INFO:tensorflow:global_step/sec: 177.236
INFO:tensorflow:loss = 40772.7, step = 228701 (0.564 sec)
INFO:tensorflow:global_step/sec: 177.484
INFO:tensorflow:loss = 36273.9, step = 228801 (0.563 sec)
INFO:tensorflow:global_step/sec: 135.17
INFO:tensorflow:loss = 37892.6, step = 228901 (0.742 sec)
INFO:tensorflow:global_step/sec: 137.5
INFO:tensorflow:loss = 34574.9, step = 229001 (0.725 sec)
INFO:tensorflow:global_step/sec: 162.722
INFO:tensorfl

INFO:tensorflow:loss = 55216.7, step = 236401 (0.723 sec)
INFO:tensorflow:global_step/sec: 168.016
INFO:tensorflow:loss = 43188.0, step = 236501 (0.596 sec)
INFO:tensorflow:global_step/sec: 170.094
INFO:tensorflow:loss = 47178.7, step = 236601 (0.587 sec)
INFO:tensorflow:global_step/sec: 169.801
INFO:tensorflow:loss = 44120.3, step = 236701 (0.589 sec)
INFO:tensorflow:global_step/sec: 181.639
INFO:tensorflow:loss = 40315.6, step = 236801 (0.550 sec)
INFO:tensorflow:global_step/sec: 182.655
INFO:tensorflow:loss = 48796.1, step = 236901 (0.548 sec)
INFO:tensorflow:global_step/sec: 178.805
INFO:tensorflow:loss = 49549.7, step = 237001 (0.559 sec)
INFO:tensorflow:global_step/sec: 173.185
INFO:tensorflow:loss = 39610.6, step = 237101 (0.578 sec)
INFO:tensorflow:global_step/sec: 180.901
INFO:tensorflow:loss = 27009.5, step = 237201 (0.552 sec)
INFO:tensorflow:global_step/sec: 182.186
INFO:tensorflow:loss = 39249.2, step = 237301 (0.549 sec)
INFO:tensorflow:global_step/sec: 154.012
INFO:tenso

## 7. Evaluate the Model

In [13]:
test_size = 5000

test_input_fn = lambda: tfrecods_input_fn(file_names= test_data_files, 
                                      mode= tf.estimator.ModeKeys.EVAL,
                                      batch_size= test_size)

estimator = create_estimator(run_config, hparams)
results = estimator.evaluate(input_fn=test_input_fn, steps=1)

print("")
print(results)
rmse = round(math.sqrt(results["average_loss"]),5)
print("")
print("RMSE: {}".format(rmse))

INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11b8d4ba8>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': 'trained_models/reg-model-05'}
INFO:tensorflow:Starting evaluation at 2017-10-31-09:19:37
INFO:tensorflow:Restoring parameters from trained_models/reg-model-05/model.ckpt-240000
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2017-10-31-09:19:37
INFO:tensorflow:Saving dict for global step 240000: average_loss = 94.0807, global_step = 240000, loss = 470404.0


## 8. Prediction

In [14]:
import itertools

predict_input_fn = lambda: tfrecods_input_fn(file_names= test_data_files, 
                                      mode= tf.estimator.ModeKeys.PREDICT,
                                      batch_size= 5)

predictions = estimator.predict(input_fn=predict_input_fn)
print("")
print(list(itertools.islice(predictions, 5)))


INFO:tensorflow:Restoring parameters from trained_models/reg-model-05/model.ckpt-240000
[{'predictions': array([ 50.97230148], dtype=float32)}, {'predictions': array([-3.95964074], dtype=float32)}, {'predictions': array([ 17.57191086], dtype=float32)}, {'predictions': array([ 3.14516091], dtype=float32)}, {'predictions': array([ 3.19907188], dtype=float32)}]
